In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from prepare_data import prepare

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
test['revenue'] = np.nan

In [3]:
train = pd.merge(train, pd.read_csv('data/trainRatingTotalVotes.csv'), how='left', on=['imdb_id'])
test = pd.merge(test, pd.read_csv('data/testRatingTotalVotes.csv'), how='left', on=['imdb_id'])

trainV2 = pd.read_csv('data/trainV2.csv')

In [73]:
train = pd.concat([train, trainV2])

In [3]:
train['revenue'] = np.log1p(train['revenue'])
y = train['revenue'].values

json_cols = ['genres', 'production_companies', 'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

In [4]:
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

In [5]:
for col in tqdm(json_cols + ['belongs_to_collection']) :
    train[col] = train[col].apply(lambda x : get_dictionary(x))
    test[col] = test[col].apply(lambda x : get_dictionary(x))

100%|████████████████████████████████████████████| 8/8 [00:08<00:00,  1.05s/it]


In [6]:
def get_json_dict(df) :
    global json_cols
    result = dict()
    for e_col in json_cols :
        d = dict()
        rows = df[e_col].values
        for row in rows :
            if row is None : continue
            for i in row :
                if i['name'] not in d :
                    d[i['name']] = 0
                d[i['name']] += 1
        result[e_col] = d
    return result

In [7]:
train_dict = get_json_dict(train)
test_dict = get_json_dict(test)

In [8]:
for col in json_cols :
    
    remove = []
    train_id = set(list(train_dict[col].keys()))
    test_id = set(list(test_dict[col].keys()))   
    
    remove += list(train_id - test_id) + list(test_id - train_id)
    for i in train_id.union(test_id) - set(remove) :
        if train_dict[col][i] < 10 or i == '' :
            remove += [i]
            
    for i in remove :
        if i in train_dict[col] :
            del train_dict[col][i]
        if i in test_dict[col] :
            del test_dict[col][i]

In [9]:
all_data = prepare(pd.concat([train, test]).reset_index(drop=True), train_dict)

train = all_data.loc[:train.shape[0] -1,:]
test = all_data.loc[train.shape[0]:,:] 